# MODEL TensorFlow Keras

In [9]:
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow import keras



In [10]:
current_dir = os.getcwd()

evalDir = os.path.join(current_dir, 'data/Evaluation_Set')
trainDir = os.path.join(current_dir, 'data/Training_Set')
testDir = os.path.join(current_dir, 'data/Test_Set')

# Define data directories 
df_test = pd.read_csv(testDir+'/RFMiD_Testing_Labels.csv')
print(df_test.head())
df_train = pd.read_csv(trainDir+ '/RFMiD_Training_Labels.csv')
print(df_train.head())

   ID  Disease_Risk  DR  ARMD  MH  DN  MYA  BRVO  TSLN  ERM  ...  CME  PTCR  \
0   1             1   1     0   0   0    0     0     1    0  ...    0     0   
1   2             1   1     0   0   0    0     1     0    0  ...    0     0   
2   3             1   1     0   0   0    0     0     0    0  ...    0     0   
3   4             1   0     0   0   0    0     0     0    0  ...    0     0   
4   5             1   0     0   0   0    0     0     0    0  ...    0     0   

   CF  VH  MCA  VS  BRAO  PLQ  HPED  CL  
0   0   0    0   0     0    0     0   0  
1   0   0    0   0     0    0     0   0  
2   0   0    0   0     0    0     0   0  
3   0   0    0   0     0    0     0   0  
4   0   0    0   0     0    0     0   0  

[5 rows x 47 columns]
   ID  Disease_Risk  DR  ARMD  MH  DN  MYA  BRVO  TSLN  ERM  ...  CME  PTCR  \
0   1             1   1     0   0   0    0     0     0    0  ...    0     0   
1   2             1   1     0   0   0    0     0     0    0  ...    0     0   
2   3        

## Data Preparation

In [11]:
df_train['ID'] = df_train['ID'].astype(str) + '.png'
df_test['ID'] = df_test['ID'].astype(str) + '.png'


In [12]:
# Load and preprocess images
image_size = (224, 224)  # Set your desired image size
batch_size = 32

datagen = ImageDataGenerator(rescale=1.0/255.0)  # Normalize pixel values

train_data = datagen.flow_from_dataframe(
    df_train, 
    directory=trainDir + "/Training/", 
    x_col="ID", 
    y_col="Disease_Risk", 
    target_size=image_size, 
    batch_size=batch_size, 
    class_mode='raw',
    shuffle=True,
    seed=42,

)

test_data = datagen.flow_from_dataframe(
    df_test, 
    directory=testDir + "/Test/", 
    x_col="ID", 
    y_col="Disease_Risk", 
    target_size=image_size, 
    batch_size=batch_size, 
    class_mode='raw',
    shuffle=True,
    seed=42,

)

Found 1920 validated image filenames.
Found 640 validated image filenames.


In [13]:
# Define the Keras model
model = keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


# Train the model
history = model.fit(train_data, epochs=10, validation_data=test_data)

Epoch 1/10
60/60 [==============================] - 256s 4s/step - loss: 0.6185 - accuracy: 0.7714 - val_loss: 0.4050 - val_accuracy: 0.8391
Epoch 2/10
60/60 [==============================] - 251s 4s/step - loss: 0.4386 - accuracy: 0.8224 - val_loss: 0.4262 - val_accuracy: 0.8391
Epoch 3/10
60/60 [==============================] - 254s 4s/step - loss: 0.4351 - accuracy: 0.8224 - val_loss: 0.3679 - val_accuracy: 0.8375
Epoch 4/10
60/60 [==============================] - 3094s 52s/step - loss: 0.4171 - accuracy: 0.8276 - val_loss: 0.3422 - val_accuracy: 0.8422
Epoch 5/10
60/60 [==============================] - 249s 4s/step - loss: 0.4304 - accuracy: 0.8292 - val_loss: 0.3531 - val_accuracy: 0.8531
Epoch 6/10
60/60 [==============================] - 247s 4s/step - loss: 0.4120 - accuracy: 0.8276 - val_loss: 0.3595 - val_accuracy: 0.8406
Epoch 7/10
60/60 [==============================] - 246s 4s/step - loss: 0.4269 - accuracy: 0.8255 - val_loss: 0.3648 - val_accuracy: 0.8500
Epoch 8/10


In [14]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_data)
print(f'Test accuracy: {test_acc * 100:.2f}%')
print(f'Test loss: {test_loss:.2f}')

20/20 [==============================] - 66s 3s/step - loss: 0.3480 - accuracy: 0.8719
Test accuracy: 87.19%
Test loss: 0.35


In [15]:
# Predict labels for test images
predictions = model.predict(test_data)

# Assuming you have the list of column names from your dataset
column_names = df_train.columns[1:]

# Print the predictions for each test image
for i, prediction in enumerate(predictions):
    predicted_labels = [int(round(value)) for value in prediction]  # Convert predicted probabilities to 0 or 1
    predicted_labels_dict = dict(zip(column_names, predicted_labels))
    print(f"Predictions for Test Image {i+1}:")
    print(predicted_labels_dict)
    print()

20/20 [==============================] - 66s 3s/step
Predictions for Test Image 1:
{'Disease_Risk': 0}

Predictions for Test Image 2:
{'Disease_Risk': 1}

Predictions for Test Image 3:
{'Disease_Risk': 0}

Predictions for Test Image 4:
{'Disease_Risk': 0}

Predictions for Test Image 5:
{'Disease_Risk': 1}

Predictions for Test Image 6:
{'Disease_Risk': 1}

Predictions for Test Image 7:
{'Disease_Risk': 1}

Predictions for Test Image 8:
{'Disease_Risk': 1}

Predictions for Test Image 9:
{'Disease_Risk': 1}

Predictions for Test Image 10:
{'Disease_Risk': 0}

Predictions for Test Image 11:
{'Disease_Risk': 1}

Predictions for Test Image 12:
{'Disease_Risk': 1}

Predictions for Test Image 13:
{'Disease_Risk': 1}

Predictions for Test Image 14:
{'Disease_Risk': 1}

Predictions for Test Image 15:
{'Disease_Risk': 1}

Predictions for Test Image 16:
{'Disease_Risk': 1}

Predictions for Test Image 17:
{'Disease_Risk': 0}

Predictions for Test Image 18:
{'Disease_Risk': 1}

Predictions for Test